In [1]:
### IMPORTS

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
### FUNCTIONS

## Min-Max Normalization
# X: array to be normalized
def normalize(X):
    X_norm = (X-np.min(X, axis=0))/(np.max(X, axis=0) - np.min(X, axis=0))
    return X_norm

## Univariate Gradient Descent
# X: feature vector
# y: target vector
# alpha: step size
# max_iter: number of iterations 
def uni_gradient_descent(X, y, alpha, max_iter):
    m = 1
    b = 1
    n = len(X)
    for i in range(max_iter):
        m_gradient = np.mean(-2*X*(y - (m*X + b)))
        b_gradient = np.mean(-2*(y - (m*X + b)))
        m = m - alpha*m_gradient
        b = b - alpha*b_gradient
    return m, b

### Multivariate Gradient Descent
# X: feature array 
# y: target vector
# alpha: step size
# max_iter: number of iterations 
def multi_gradient_descent(X, y, alpha, max_iter):
    if (X.ndim == 1):
        X = X.reshape(1, -1)
    n, num_feats = X.shape
    m = np.ones((num_feats,1))
    b = 1
    for i in range(max_iter):
        m_gradient = np.sum(-2*(y - (X@m + b))*X, axis=0).reshape(1, -1).transpose()
        b_gradient = np.sum(-2*(y - (X@m + b)))
        m = m - alpha*m_gradient/n
        b = b - alpha*b_gradient/n
    return m, b

In [3]:
### PREPROCESSING I

# read data set
data = pd.read_csv(r"C:\Users\kingh\Downloads\Concrete_Data.csv")

# extract feature variables
X = data.iloc[:,:8].to_numpy()
X_test = X[501:631, :]
X_train = np.vstack((X[0:501, :], X[631:, :]))

# extract target variable
y = data.iloc[:, -1].to_numpy().reshape((-1,1))
y_test = y[501:631]
y_train = np.vstack((y[0:501], y[631:]))

In [4]:
### PREPROCESSING II

# create normalized X
X_train_norm = normalize(X_train)
X_test_norm = normalize(X_test)

# UNIVARIATE LINEAR MODELS

## NORMALIZED PREDICTORS

In [5]:
### GRID SEARCH FOR OPTIMAL HYPERPARAMS (UNIVARIATE, NORMALIZED)

alpha_values = np.logspace(-3, -1, 10)
max_iter_values = np.logspace(1, 3, 10, dtype=int)
best_m = None
best_b = None
best_VE_train = None
best_params = None
best_VE = None
best_avg_VE = -np.inf

for alpha in alpha_values:
    for max_iter in max_iter_values:
        
        # initialize parameters
        num_feats = len(X_train_norm[0])          # number of features
        
        # conduct gradient descent to obtain a univariate linear regression model for each feature
        m = np.zeros(num_feats)
        b = np.zeros(num_feats)
        for i in range(num_feats):
            X_i = X_train_norm[:, i].reshape((-1,1))
            m_i, b_i = uni_gradient_descent(X_i, y_train, alpha, max_iter)
            m[i] = m_i
            b[i] = b_i
        
        # evaluate variances
        var_train = np.var(y_train)
        var_test = np.var(y_test)
        
        # model predictions
        m = m.reshape(-1,1)
        b = b.reshape(-1,1)
        y_train_pred = (X_train_norm.transpose()*m + b).transpose()
        y_test_pred = (X_test_norm.transpose()*m + b).transpose()
        
        # train + test MSE
        train_MSE = np.mean((y_train_pred - np.tile(y_train, (1, num_feats)))**2, axis=0)
        test_MSE  = np.mean((y_test_pred - np.tile(y_test, (1, num_feats)))**2, axis=0)
        
        # train + test VE
        train_VE = 1 - (train_MSE / var_train)
        test_VE  = 1 - (test_MSE / var_test)
        
        # update best model based on average test VE
        if np.mean(test_VE) > best_avg_VE:
            best_avg_VE = np.mean(test_VE)
            best_VE = test_VE
            best_params = [alpha, max_iter]
            best_m = m
            best_b = b
            best_VE_train = train_VE

print("Best Parameters:", best_params)
print("Best Test VE:", best_VE)
print("Corresponding Train VE:", best_VE_train)
print("m:", best_m)
print("b:", best_b)

Best Parameters: [0.004641588833612777, 129]
Best Test VE: [ 0.18619992 -0.05296291  0.13085162 -0.11010294  0.14396512 -0.23920968
 -0.11501515  0.01860353]
Corresponding Train VE: [-0.10267314 -0.32989611 -0.42359578 -0.33249195 -0.31981907 -0.30055846
 -0.32186009 -0.36360203]
m: [[12.09544269]
 [ 7.46018168]
 [ 6.11105546]
 [10.40240548]
 [ 7.18534966]
 [11.12862652]
 [10.22574434]
 [ 5.13395483]]
b: [[23.73218639]
 [25.21405681]
 [25.21129053]
 [23.6057197 ]
 [25.39393915]
 [23.25085355]
 [23.78495704]
 [25.84555499]]


In [31]:
### TRAIN + ANALYZE UNIVARIATE MODELS (NORMALIZED)

# initialize parameters
num_feats = len(X_train_norm[0])          # number of features
alpha = 0.005                              # learning rate       
max_iter = 130                            # number of iterations

# conduct gradient descent to obtain a univariate linear regression model for each feature
m = np.zeros(num_feats)
b = np.zeros(num_feats)
for i in range(num_feats):
    X_i = X_train_norm[:, i].reshape((-1,1))
    m_i, b_i = uni_gradient_descent(X_i, y_train, alpha, max_iter)
    m[i] = m_i
    b[i] = b_i

# evaluate variances
var_train = np.var(y_train)
var_test = np.var(y_test)

# model predictions
m = m.reshape(-1,1)
b = b.reshape(-1,1)
y_train_pred = (X_train_norm.transpose()*m + b).transpose()
y_test_pred = (X_test_norm.transpose()*m + b).transpose()

# train + test MSE
train_MSE = np.mean((y_train_pred - np.tile(y_train, (1, num_feats)))**2, axis=0)
test_MSE  = np.mean((y_test_pred - np.tile(y_test, (1, num_feats)))**2, axis=0)

# train + test VE
train_VE = 1 - (train_MSE / var_train)
test_VE  = 1 - (test_MSE / var_test)

print("m:", m)
print("b:", b)
print("Test VE:", test_VE)
print("Train VE:", train_VE)
print("Test MSE:", test_MSE)
print("Train MSE:", train_MSE)

m: [[12.56225204]
 [ 7.71724786]
 [ 6.2155356 ]
 [10.67385511]
 [ 7.46990294]
 [11.41878722]
 [10.50429539]
 [ 5.34664105]]
b: [[24.54717439]
 [26.18380807]
 [26.19291599]
 [24.43239196]
 [26.37837713]
 [24.0447017 ]
 [24.62620564]
 [26.87523594]]
Test VE: [ 0.17238119 -0.04683249  0.13978374 -0.12812279  0.15721965 -0.25756707
 -0.12567177  0.02813135]
Train VE: [-0.04112086 -0.25962091 -0.35554444 -0.27968436 -0.24648718 -0.25074039
 -0.26707893 -0.28638303]
Test MSE: [180.2050573  227.93646867 187.30280136 245.63655401 183.50632011
 273.82164925 245.10287164 211.61390238]
Train MSE: [288.47410752 349.0161713  375.59469483 354.57535927 345.3770724
 346.55555477 351.08264145 356.43142684]


## RAW PREDICTORS

In [189]:
### GRID SEARCH FOR OPTIMAL HYPERPARAMS (UNIVARIATE, RAW)

alpha_values = [0.0001, 0.00001]
max_iter_values = np.logspace(1, 3, 5, dtype=int)
best_m = None
best_b = None
best_VE_train = None
best_params = None
best_VE = None
best_avg_VE = -np.inf

for alpha in alpha_values:
    for max_iter in max_iter_values:
        # initialize parameters
        num_feats = len(X_train[0])          # number of features
        
        # conduct gradient descent to obtain a univariate linear regression model for each feature
        m = np.zeros(num_feats)
        b = np.zeros(num_feats)
        for i in range(num_feats):
            X_i = X_train[:, i].reshape((-1,1))
            m_i, b_i = uni_gradient_descent(X_i, y_train, alpha, max_iter)
            m[i] = m_i
            b[i] = b_i
        
        # evaluate variances
        var_train = np.var(y_train)
        var_test = np.var(y_test)
        
        # model predictions
        m = m.reshape(-1,1)
        b = b.reshape(-1,1)
        y_train_pred = (X_train.transpose()*m + b).transpose()
        y_test_pred = (X_test.transpose()*m + b).transpose()
        
        # train + test MSE
        train_MSE = np.mean((y_train_pred - np.tile(y_train, (1, num_feats)))**2, axis=0)
        test_MSE  = np.mean((y_test_pred - np.tile(y_test, (1, num_feats)))**2, axis=0)
                
        # train + test VE
        train_VE = 1 - (train_MSE / var_train)
        test_VE  = 1 - (test_MSE / var_test)
        if (test_VE[7]+train_VE[7])/2 > best_avg_VE:
            best_avg_VE = test_VE[7]
            best_VE = test_VE
            best_params = [alpha, max_iter]
            best_m = m
            best_b = b
            best_VE_train = train_VE

print(best_avg_VE)
print("Best Parameters:", best_params)
print("Best Test VE:", best_VE)
print("Corresponding Train VE:", best_VE_train)
print("m:", best_m)
print("b:", best_b)

C:\Users\kingh\AppData\Local\Temp\ipykernel_32256\2824116174.py:37: RuntimeWarning: overflow encountered in square
  train_MSE = np.mean((y_train_pred - np.tile(y_train, (1, num_feats)))**2, axis=0)
C:\Users\kingh\AppData\Local\Temp\ipykernel_32256\2824116174.py:38: RuntimeWarning: overflow encountered in square
  test_MSE  = np.mean((y_test_pred - np.tile(y_test, (1, num_feats)))**2, axis=0)
C:\Users\kingh\AppData\Local\Temp\ipykernel_32256\1696712276.py:21: RuntimeWarning: invalid value encountered in double_scalars
  m = m - alpha*m_gradient


-2.2023126630627305
Best Parameters: [0.0001, 1000]
Best Test VE: [        nan        -inf -1.04135567         nan -1.35068735         nan
         nan -2.20231266]
Corresponding Train VE: [        nan        -inf -2.40337679         nan -1.05602793         nan
         nan -1.66191696]
m: [[            nan]
 [1.10562839e+196]
 [2.17041825e-001]
 [            nan]
 [3.15501148e+000]
 [            nan]
 [            nan]
 [3.12945023e-001]]
b: [[            nan]
 [6.54918240e+193]
 [5.09634067e+000]
 [            nan]
 [3.63294892e+000]
 [            nan]
 [            nan]
 [4.78737552e+000]]


In [182]:
print(alpha_values)
-0.5742146245315389 -0.07310492
# -0.28685476121797904 [9.651254005555555e-07, 31] # coarse aggregate
# -0.5233298770915911 [1.5777402395e-06, 10000] # fine aggregate
# -0.2900672144550507 [0.0001, 15000] # age
# -2.571625288578917 [7.554e-05, 31] # blast furnace slag
# -0.5249004344951762 [2.9e-05, 500] # water

[1.29154967e-07 3.00148920e-07 4.71142872e-07 6.42136825e-07
 8.13130777e-07 9.84124730e-07 1.15511868e-06 1.32611263e-06
 1.49710659e-06 1.66810054e-06]


-0.647319544531539

In [190]:
### TRAIN + ANALYZE UNIVARIATE MODELS (RAW)

# initialize parameters
num_feats = len(X_train[0])              # number of features
alpha = 0.0001                              # learning rate       
max_iter = 15000                            # number of iterations

# conduct gradient descent to obtain a univariate linear regression model for each feature
m = np.zeros(num_feats)
b = np.zeros(num_feats)
for i in range(num_feats):
    X_i = X_train[:, i].reshape((-1,1))
    m_i, b_i = uni_gradient_descent(X_i, y_train, alpha, max_iter)
    m[i] = m_i
    b[i] = b_i

# evaluate variances
var_train = np.var(y_train)
var_test = np.var(y_test)

# model predictions
m = m.reshape(-1,1)
b = b.reshape(-1,1)
y_train_pred = (X_train.transpose()*m + b).transpose()
y_test_pred = (X_test.transpose()*m + b).transpose()

# train + test MSE
train_MSE = np.mean((y_train_pred - np.tile(y_train, (1, num_feats)))**2, axis=0)
test_MSE  = np.mean((y_test_pred - np.tile(y_test, (1, num_feats)))**2, axis=0)

# train + test VE
train_VE = 1 - (train_MSE / var_train)
test_VE  = 1 - (test_MSE / var_test)

print("m", m)
print("b", b)
print("Test VE:", test_VE)
print("Train VE:", train_VE)
print("Test MSE:", test_MSE)
print("Train MSE:", train_MSE)

C:\Users\kingh\AppData\Local\Temp\ipykernel_32256\1696712276.py:21: RuntimeWarning: invalid value encountered in double_scalars
  m = m - alpha*m_gradient
C:\Users\kingh\AppData\Local\Temp\ipykernel_32256\1696712276.py:19: RuntimeWarning: invalid value encountered in multiply
  m_gradient = np.mean(-2*X*(y - (m*X + b)))
C:\Users\kingh\AppData\Local\Temp\ipykernel_32256\1696712276.py:20: RuntimeWarning: invalid value encountered in multiply
  b_gradient = np.mean(-2*(y - (m*X + b)))


m [[       nan]
 [       nan]
 [0.00383079]
 [       nan]
 [1.56875327]
 [       nan]
 [       nan]
 [0.13139059]]
b [[        nan]
 [        nan]
 [32.65221781]
 [        nan]
 [22.675146  ]
 [        nan]
 [        nan]
 [27.9536373 ]]
Test VE: [        nan         nan -0.09774375         nan  0.25866472         nan
         nan -0.29006721]
Train VE: [        nan         nan -0.06695411         nan -0.00486347         nan
         nan  0.072547  ]
Test MSE: [         nan          nan 239.02184727          nan 161.4177514
          nan          nan 280.89820414]
Train MSE: [         nan          nan 295.63199279          nan 278.4278962
          nan          nan 256.97897737]


## MULTIVARIATE LINEAR MODELS

## NORMALIZED PREDICTORS

In [26]:
### GRID SEARCH FOR OPTIMAL HYPERPARAMS (MULTIVARIATE, NORMALIZED)

alpha_values = [0.021544346900318832, 0.021544346900318832]
max_iter_values = np.logspace(3, 5, 10, dtype=int)
best_m = None
best_b = None
best_VE_train = None
best_params = None
best_VE = -np.inf

for alpha in alpha_values:
    for max_iter in max_iter_values:
        
        # conduct gradient descent to obtain a multivariate linear regression model using all features
        m, b = multi_gradient_descent(X_train_norm, y_train, alpha, max_iter)
        
        # evaluate variances
        var_train = np.var(y_train)
        var_test = np.var(y_test)
        
        # model predictions
        y_train_pred = X_train_norm@m + b
        y_test_pred = X_test_norm@m + b
        
        # train + test MSE
        train_MSE = mean_squared_error(y_train, y_train_pred)
        test_MSE  = mean_squared_error(y_test, y_test_pred)
        
        # train + test VE
        train_VE = 1 - (train_MSE / var_train)
        test_VE  = 1 - (test_MSE / var_test)
        if test_VE > best_VE:
            best_VE = test_VE
            best_params = [alpha, max_iter]
            best_m = m
            best_b = b
            best_VE_train = train_VE

print("Best Parameters:", best_params)
print("Best Test VE:", best_VE)
print("Corresponding Train VE:", best_VE_train)
print("m:", best_m)
print("b:", best_b)

Best Parameters: [0.021544346900318832, 1000]
Best Test VE: 0.36691753817427264
Corresponding Train VE: 0.568385102525635
m: [[ 35.03827856]
 [ 17.72233663]
 [  3.27489821]
 [-11.19919354]
 [ 19.21362897]
 [  2.22733177]
 [ -0.97891267]
 [ 26.83758829]]
b: 14.735515963644607


In [29]:
### TRAIN + ANALYZE MULTIVARIATE MODELS (NORMALIZED)

# initialize parameters
alpha = 0.022                              # learning rate       
max_iter = 1000                            # number of iterations

# conduct gradient descent to obtain a multivariate linear regression model using all features
m, b = multi_gradient_descent(X_train_norm, y_train, alpha, max_iter)

# evaluate variances
var_train = np.var(y_train)
var_test = np.var(y_test)

# model predictions
y_train_pred = X_train_norm@m + b
y_test_pred = X_test_norm@m + b

# train + test MSE
train_MSE = mean_squared_error(y_train, y_train_pred)
test_MSE  = mean_squared_error(y_test, y_test_pred)

# train + test VE
train_VE = 1 - (train_MSE / var_train)
test_VE  = 1 - (test_MSE / var_test)

print("m:", m)
print("b:", b)
print("Test VE:", test_VE)
print("Train VE:", train_VE)
print("Test MSE:", test_MSE)
print("Train MSE:", train_MSE)

m: [[ 35.18964021]
 [ 17.84399068]
 [  3.38439654]
 [-11.41243485]
 [ 19.23354567]
 [  2.22605995]
 [ -1.03417359]
 [ 27.13892915]]
b: 14.696279123791093
Test VE: 0.3672723351659948
Train VE: 0.5698961819760422
Test MSE: 137.76961600809852
Train MSE: 119.17330563402975


## RAW PREDICTORS

In [22]:
### GRID SEARCH FOR OPTIMAL HYPERPARAMS (MULTIVARIATE, RAW)

alpha_values = [3.59381366e-07, 3.59381366e-07]
max_iter_values = np.logspace(3, 5, 10, dtype=int)
best_m = None
best_b = None
best_VE_train = None
best_params = None
best_VE = -np.inf

for alpha in alpha_values:
    for max_iter in max_iter_values:
        
        # conduct gradient descent to obtain a multivariate linear regression model using all features
        m, b = multi_gradient_descent(X_train, y_train, alpha, max_iter)
        
        # evaluate variances
        var_train = np.var(y_train)
        var_test = np.var(y_test)
        
        # model predictions
        y_train_pred = X_train@m + b
        y_test_pred = X_test@m + b
        
        # train + test MSE
        train_MSE = mean_squared_error(y_train, y_train_pred)
        test_MSE  = mean_squared_error(y_test, y_test_pred)
        
        # train + test VE
        train_VE = 1 - (train_MSE / var_train)
        test_VE  = 1 - (test_MSE / var_test)
        if test_VE > best_VE:
            best_VE = test_VE
            best_params = [alpha, max_iter]
            best_m = m
            best_b = b
            best_VE_train = train_VE

print("Best Parameters:", best_params)
print("Best Test VE:", best_VE)
print("Corresponding Train VE:", best_VE_train)
print("m:", best_m)
print("b:", best_b)

Best Parameters: [3.59381366e-07, 4641]
Best Test VE: 0.5087655555166335
Corresponding Train VE: 0.5718579694046066
m: [[ 0.08774793]
 [ 0.06561048]
 [ 0.03166866]
 [-0.03871413]
 [ 0.92716067]
 [ 0.00549369]
 [-0.00615953]
 [ 0.10566376]]
b: 0.9967969828652573


In [23]:
### TRAIN + ANALYZE MULTIVARIATE MODELS (RAW)
# initialize parameters
alpha = 3.6e-07                              # learning rate       
max_iter = 4500                            # number of iterations

# conduct gradient descent to obtain a multivariate linear regression model using all features
m, b = multi_gradient_descent(X_train, y_train, alpha, max_iter)

# evaluate variances
var_train = np.var(y_train)
var_test = np.var(y_test)

# model predictions
y_train_pred = X_train@m + b
y_test_pred = X_test@m + b

# train + test MSE
train_MSE = mean_squared_error(y_train, y_train_pred)
test_MSE  = mean_squared_error(y_test, y_test_pred)

# train + test VE
train_VE = 1 - (train_MSE / var_train)
test_VE  = 1 - (test_MSE / var_test)

print("m:", m)
print("b:", b)
print("Test VE:", test_VE)
print("Train VE:", train_VE)
print("Test MSE:", test_MSE)
print("Train MSE:", train_MSE)

m: [[ 0.08770124]
 [ 0.06513533]
 [ 0.03186483]
 [-0.03387814]
 [ 0.92776233]
 [ 0.00482891]
 [-0.00636438]
 [ 0.10498659]]
b: 0.9968046353632825
Test VE: 0.5091996724879673
Train VE: 0.5699769001358561
Test MSE: 106.86647102070529
Train MSE: 119.15094022008408
